In [ ]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

In [ ]:
include("quadratic_cost.jl")
include("real_quadruped.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

In [ ]:
h = 1
v_init_z = sqrt(2 * 9.81 * h)

In [ ]:
# Dynamics model
model = RealQuadruped()

# Some parameters
dt = 0.02
N = 41
times = range(0, dt * (N - 1), length=N)
n = 37
m = 13

# initial state
xinit = zeros(n)

xinit[3] = 0.5 # zb

xinit[4] = deg2rad(15) # roll

xinit[7] = 0.15 # x1
xinit[8] = 0.1  # y1

xinit[10] = -0.15 # x2
xinit[11] = 0.1   # y2

xinit[13] = 0.15 # x3
xinit[14] = -0.1 # y3

xinit[16] = -0.15 # x4
xinit[17] = -0.1  # y4

xinit[21] = -v_init_z # vzb

# Desired final state
xterm = zeros(n)

xterm[3] = 0.3 # zb

xterm[7:17] = xinit[7:17];

In [ ]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, xterm, dt);

In [ ]:
# Objective
Q_diag = 10.0 * ones(n)
Q_diag[end] = 0.0
Q = Diagonal(Q_diag)

R_diag = 1e-3 * ones(m)
R_diag[end] = 0.0
R = Diagonal(R_diag)

Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end

push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [ ]:
# Define the NLP
nlp = HybridNLP(model, obj, N, xinit, xterm);

In [ ]:
# initialize guess
Xguess = [zeros(n) for x in Xref]

for k = 1:N
    Xguess[k][1:36] = xterm[1:36]
    Xguess[k][end] = dt * (k - 1)
end

mb, g = model.mb, model.g
Uguess = [zeros(m) for u in Uref]

for k = 1:N-1
    Uguess[k][3] = -mb*g/4
    Uguess[k][6] = -mb*g/4
    Uguess[k][9] = -mb*g/4
    Uguess[k][12] = -mb*g/4
    Uguess[k][end] = dt;
end

Z0 = packZ(nlp, Xguess, Uguess);

In [ ]:
Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

In [ ]:
data_idx = 3

if isdir("../results/data_" * string(data_idx)) == false
    mkdir("../results/data_" * string(data_idx))
end

writedlm("../results/data_" * string(data_idx) * "/data_" * string(data_idx) * ".csv", Z_sol, ',')